In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect(
                database='postgres_db',
                user='postgres_user',
                password='admin',
                host='client.tuple-mia.com',
                port='5432',
                sslmode='require'
            )

In [3]:
temp = conn.cursor()

In [4]:
import yaml
with open('/home/ubuntu/MappingBuffer_event.yml', 'r') as f:
    doc = yaml.load(f)

In [5]:
doc['table_map']

{'CUSTOMER_CONTACT': 'contact',
 'CUSTOMER_MASTER': 'users',
 'CUSTOMER_SECONDARY': 'temp_customer_secondary',
 'EVENT_LOG': 'event',
 'EVENT_MASTER': 'event_master',
 'TRANSACTION_MASTER': 'trans'}

In [6]:
doc['column_map']

{'CUSTOMER_CONTACT': {'cust_id': 'id',
  'email_id': 'email',
  'facebook_id': 'facebook_id',
  'firstname': 'first_name',
  'lastname': 'last_name',
  'phone_number': 'phone'},
 'CUSTOMER_MASTER': {'birthday': 'birthday',
  'create_date': 'create_date',
  'cust_id': 'id'},
 'CUSTOMER_SECONDARY': {'cust_id': 'cust_id'},
 'EVENT_LOG': {'action_type': 'actiontype',
  'cust_id': 'from_id',
  'product_id': 'to_id',
  'timestamp': 'timestamp'},
 'EVENT_MASTER': {'action_category_id': None,
  'action_category_name': None,
  'action_name': 'action_desc',
  'action_type': 'actiontype'},
 'TRANSACTION_MASTER': {'cust_id': 'userid',
  'product_id': 'product_id',
  'quantity': 'quantity',
  'renewal': 'renewal',
  'revenue': 'revenue',
  'timestamp': 'purchase_date'}}

In [7]:
query = "SELECT * FROM %s where %s = ( SELECT %s FROM %s WHERE %s = 'like')"%(
        doc['table_map']['EVENT_LOG'],
        doc['column_map']['EVENT_LOG']['action_type'],
        doc['column_map']['EVENT_MASTER']['action_type'],
        doc['table_map']['EVENT_MASTER'],
        doc['column_map']['EVENT_MASTER']['action_name']        
    )

In [8]:
print query

SELECT * FROM event where actiontype = ( SELECT actiontype FROM event_master WHERE action_desc = 'like')


In [9]:
temp.execute(query)

In [10]:
event = temp.fetchall()

In [11]:
len(event)

2075810

In [12]:
colnames = [desc[0] for desc in temp.description]

In [13]:
colnames

['to_id', 'timestamp', 'actiontype', 'from_id']

In [14]:
import pandas as pd

In [15]:
event_df = pd.DataFrame(event, columns=colnames)

In [16]:
event_df.head()

,to_id,timestamp,actiontype,from_id
0,255074,2016-11-19 00:52:13+00:00,1,6148771
1,304603,2016-11-20 08:03:32+00:00,1,6148771
2,368895,2016-11-15 09:28:35+00:00,1,6148771
3,376784,2016-11-15 09:43:29+00:00,1,6148771
4,462976,2016-11-15 09:44:27+00:00,1,6148771


In [17]:
event_df.rename(columns={doc['column_map']['EVENT_LOG']['cust_id']:'cust_id'}, inplace = True)
event_df.rename(columns={doc['column_map']['EVENT_LOG']['product_id']:'product_id'}, inplace = True)
event_df.rename(columns={doc['column_map']['EVENT_LOG']['action_type']:'actiontype'}, inplace = True)

In [18]:
event_data = event_df[['cust_id', 'product_id', 'actiontype']]

In [19]:
event_data.head()

,cust_id,product_id,actiontype
0,6148771,255074,1
1,6148771,304603,1
2,6148771,368895,1
3,6148771,376784,1
4,6148771,462976,1


In [21]:
event_data.to_csv('/home/ubuntu/client_demo/event/event_action_header', header=True)

In [ ]:
type(event_data)

In [ ]:
import findspark

In [ ]:
findspark.init()

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext,Row
from pyspark.mllib.recommendation import ALS

In [ ]:
sc = SparkContext(appName = "BuildProductRecommendations_event")

In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
n_prod_views_rdd = sqlContext.createDataFrame(event_data).rdd

In [ ]:
n_prod_views_rdd.take(5)

In [ ]:
training_rdd, test_rdd = n_prod_views_rdd.randomSplit([8,2], 1345)

In [ ]:
training_rdd.distinct().count()

In [ ]:
test_for_predict_rdd = test_rdd.map(lambda x : (x[0], x[1]))

In [ ]:
import math

In [ ]:
seed = 49247
iterations = 10
lambdas = [0.01, 0.1]
ranks = [16]
alphas = [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 40.0, 80.0]
errors = [0 for x in range(len(alphas) * len(ranks) * len(lambdas))]
err_index = 0

In [ ]:
for lambda_ in  lambdas:
    for rank in ranks:
        for alpha in alphas:
            model = ALS.trainImplicit(training_rdd, rank, seed=seed, iterations=iterations,
                                           lambda_ = lambda_, alpha = alpha)
            predictions = model.predictAll(test_for_predict_rdd).map(lambda r : ((r[0], r[1]), r[2]))
            views_and_preds = test_rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
            error = math.sqrt(views_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
            errors[err_index] = error
            err_index += 1
            print('For rank {0} at alpha: {1} and lambda: {2}, the RMSE is{3}'.format(rank, alpha, lambda_, error))

In [ ]:
model.recommendProducts(6253829,10)